In [66]:
import sys
import os
sys.path.append('../src/util/')
import numpy as np
import pandas as pd

from general import (IGNLidarProcessor, Sentinel2Reader,
                     load_dem_utm, PcdGenerator,
                     PointCloudHandler, PcdFilter)


In [67]:
input_file = "../src/dem-data/LHD_FXX_0982_6524_PTS_C_LAMB93_IGN69.copc.laz"
sample_fraction = 0.2

processor = IGNLidarProcessor(input_file)
processor.read_point_cloud()
processor.transform_coordinates()
processor.downsample(sample_fraction=sample_fraction)
processor.generate_color()
df_ign = processor.df
df_ign.head()

Point cloud downsampled with sample fraction 0.2


,x,y,z,classification,color
259626,315601.754841,5.069443e+06,1603.25,3,"(31, 119, 180)"
2419389,315617.506961,5.069177e+06,1677.62,5,"(255, 127, 14)"
17900936,315742.764659,5.069129e+06,1720.02,5,"(255, 127, 14)"
3666107,315680.390416,5.069146e+06,1699.57,5,"(255, 127, 14)"
20740605,315874.449597,5.069438e+06,1800.28,2,"(214, 39, 40)"


In [68]:
token = os.environ.get('hdb_token')
product_path = "/home/ubuntu/project/destine-godot-mvp/src/sentinel2-data/T32TLR_20241030T103151_TCI_20m.jp2"
reader = Sentinel2Reader(filepath=product_path, preprocess=True)
bounds = reader.bounds
width = reader.width
height = reader.height
parameter = 'dem'

In [69]:
dem_data = load_dem_utm(token, parameter, bounds, width, height)
# Initialize and generate point cloud
pcd_gen = PcdGenerator(sat_data=reader.data,
                       dem_data=dem_data,
                       sample_fraction=sample_fraction)
pcd_gen.generate_point_cloud()
# pcd_gen.downsample(sample_fraction=sample_fraction)
df = pcd_gen.df
df.head()

Total points before sampling: 30140100


,x,y,z,color
0,300010,4990210,2474.875,"(116, 95, 79)"
1,300030,4990210,2468.250,"(112, 91, 73)"
2,300050,4990210,2459.250,"(128, 107, 88)"
3,300070,4990210,2448.500,"(139, 113, 94)"
4,300090,4990210,2435.250,"(143, 123, 104)"


In [70]:
filterer = PcdFilter(df=df, df_target=df_ign)
filterer.set_bounding_box_from_target(margin=4)
df_filtered = filterer.filter_data()
pcd = filterer.concatenate_dataframes([df_filtered, df_ign])

In [71]:
handler = PointCloudHandler(pcd)
handler.to_open3d()
handler.save_point_cloud(filename="final_IGN_LiDAR_HD.ply")

Point cloud saved to final_IGN_LiDAR_HD.ply


In [72]:
import folium
from pyproj import Transformer

# Initialize transformer to convert UTM (EPSG:32632) to WGS84 (EPSG:4326)
transformer_to_wgs84 = Transformer.from_crs("epsg:32632", "epsg:4326", always_xy=True)

def get_bbox_coordinates(df):
    """Calculate bounding box (min/max) and convert UTM to WGS84."""
    x_min, x_max = df["x"].min(), df["x"].max()
    y_min, y_max = df["y"].min(), df["y"].max()

    lat_min, lon_min = transformer_to_wgs84.transform(x_min, y_min)
    lat_max, lon_max = transformer_to_wgs84.transform(x_max, y_max)
    return (lat_min, lon_min), (lat_max, lon_max), transformer_to_wgs84.transform((x_min + x_max) / 2, (y_min + y_max) / 2)

# Get bounding boxes and map center
bbox1_min, bbox1_max, center1 = get_bbox_coordinates(df)
bbox2_min, bbox2_max, center2 = get_bbox_coordinates(df_ign)

# Define map center as the midpoint between the two dataset centers
map_center = [(center1[1] + center2[1]) / 2, (center1[0] + center2[0]) / 2]

# Initialize Folium map
m = folium.Map(location=map_center, zoom_start=8, tiles='OpenStreetMap')

# Add first bounding box (df)
folium.Rectangle(
    bounds=[bbox1_min[::-1], bbox1_max[::-1]],  # Reverse (lat, lon) order
    color="blue",
    weight=2,
    fill=True,
    fill_opacity=0.3,
    popup="Sentinel 2"
).add_to(m)

# Add second bounding box (df_ign)
folium.Rectangle(
    bounds=[bbox2_min[::-1], bbox2_max[::-1]],  # Reverse (lat, lon) order
    color="red",
    weight=2,
    fill=True,
    fill_opacity=0.3,
    popup="IGN "
).add_to(m)
m

In [93]:
filterer = PcdFilter(df=df, df_target=df_ign)
filterer.set_bounding_box_from_target(margin=6)
df_filtered = filterer.filter_data()
pcd = filterer.concatenate_dataframes([df_filtered, df_ign])

In [94]:
handler = PointCloudHandler(pcd)
handler.to_open3d()
handler.save_point_cloud(filename="IGN_LiDAR_HD.ply")

Point cloud saved to IGN_LiDAR_HD.ply


## Process all the data for a given Tile

In [108]:
import geopandas as gpd

shapefile_path = "/home/ubuntu/project/destine-godot-mvp/src/ign-resources/TA_diff_pkk_lidarhd_classe.shp"
# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file(shapefile_path)
gdf.head()

,nom_pkk,url_telech,geometry
0,LHD_FXX_0230_6852_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((230000 6851000, 230000 6852000, 2310..."
1,LHD_FXX_0232_6870_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((232000 6869000, 232000 6870000, 2330..."
2,LHD_FXX_0187_6874_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((187000 6873000, 187000 6874000, 1880..."
3,LHD_FXX_0196_6839_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((196000 6838000, 196000 6839000, 1970..."
4,LHD_FXX_0229_6846_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((229000 6845000, 229000 6846000, 2300..."


In [106]:
reader = Sentinel2Reader(filepath=product_path, preprocess=True)
bounds = reader.bounds
from shapely.geometry import box
bounding_box = box(bounds.left, bounds.bottom, bounds.right, bounds.top)

# Create a GeoDataFrame with EPSG:32632 (or your actual UTM zone)
gdf_utm = gpd.GeoDataFrame(geometry=[bounding_box], crs="EPSG:32632")

# Optionally, you can transform to Lambert 93 (EPSG:2154)
gd_lambert = gdf_utm.to_crs(epsg=2154)
gd_lambert

,geometry
0,"POLYGON ((1081873.964 6451770.709, 1073568.628..."


In [107]:
intersection = gdf[gdf.intersects(gd_lambert.geometry.values[0])]
intersection

,nom_pkk,url_telech,geometry
211238,LHD_FXX_0988_6554_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((988000 6553000, 988000 6554000, 9890..."
211239,LHD_FXX_1006_6547_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((1006000 6546000, 1006000 6547000, 10..."
211240,LHD_FXX_0982_6538_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((982000 6537000, 982000 6538000, 9830..."
211256,LHD_FXX_0974_6535_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((974000 6534000, 974000 6535000, 9750..."
211258,LHD_FXX_0978_6541_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((978000 6540000, 978000 6541000, 9790..."
...,...,...,...
242928,LHD_FXX_0998_6471_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((998000 6470000, 998000 6471000, 9990..."
242929,LHD_FXX_1004_6466_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((1004000 6465000, 1004000 6466000, 10..."
242930,LHD_FXX_1022_6471_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((1022000 6470000, 1022000 6471000, 10..."
242931,LHD_FXX_0998_6462_PTS_C_LAMB93_IGN69.copc.laz,https://storage.sbg.cloud.ovh.net/v1/AUTH_6323...,"POLYGON ((998000 6461000, 998000 6462000, 9990..."


In [ ]:
# Transform the coordinates from EPSG:2154 to EPSG:4326 (WGS 84)
gdf_wgs84 = intersection.to_crs(epsg=4326)

def get_bbox_coordinates(df):
    """Calculate bounding box (min/max) and convert UTM to WGS84."""
    x_min, x_max = df["x"].min(), df["x"].max()
    y_min, y_max = df["y"].min(), df["y"].max()

    lat_min, lon_min = transformer_to_wgs84.transform(x_min, y_min)
    lat_max, lon_max = transformer_to_wgs84.transform(x_max, y_max)
    return (lat_min, lon_min), (lat_max, lon_max), transformer_to_wgs84.transform((x_min + x_max) / 2, (y_min + y_max) / 2)

# Get bounding boxes and map center
bbox1_min, bbox1_max, center1 = get_bbox_coordinates(df)
bbox2_min, bbox2_max, center2 = get_bbox_coordinates(df_ign)

# Define map center as the midpoint between the two dataset centers
map_center = [(center1[1] + center2[1]) / 2, (center1[0] + center2[0]) / 2]

# Initialize Folium map
m = folium.Map(location=map_center, zoom_start=8, tiles='OpenStreetMap')

# Add first bounding box (df)
folium.Rectangle(
    bounds=[bbox1_min[::-1], bbox1_max[::-1]],  # Reverse (lat, lon) order
    color="Red",
    weight=2,
    fill=True,
    fill_opacity=0.3,
    popup="Sentinel 2"
).add_to(m)

# Loop through each geometry in your gdf_wgs84 and add its bounding box to the map
for _, row in gdf_wgs84.iterrows():
    # Get the bounding box coordinates (minx, miny, maxx, maxy) in EPSG:4326
    minx, miny, maxx, maxy = row['geometry'].bounds

    # Create a bounding box as a polygon in Folium (latitude, longitude)
    folium.Rectangle(
        bounds=[[miny, minx], [maxy, maxx]],  # (southwest, northeast)
        color="blue",  # Set the color of the bounding box
        weight=2,  # Set the line thickness
        fill=True,  # Whether the bounding box should be filled
        fill_opacity=0.2  # Set the fill opacity
    ).add_to(m)


In [100]:
m